## Libraries

In [7]:
import json
import boto3
from botocore.exceptions import ClientError
import warnings
warnings.filterwarnings("ignore")

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

## Initialize Resourcers

In [ ]:
def get_secret(secret_id, secret_key):
    return secret[secret_key]

In [ ]:
chat_api_key = get_secret("product-dev", "GPT35_API_KEY")
chat_api_endpoint = get_secret("product-dev", "GPT35_API_BASE")

embed_api_key = get_secret("product-dev", "EMBEDDING_API_KEY")
embed_api_endpoint = get_secret("product-dev", "EMBEDDING_API_BASE")

## Endpoint and API Keys from GenAI Marketplace

In [4]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI

## LLM openai
# Chat Completion
model = AzureChatOpenAI(azure_endpoint=chat_api_endpoint,
    api_key=chat_api_key,
    api_version="2023-05-15",
    azure_deployment="gpt-35-turbo-16k",
    temperature=0
)

# Embeddings
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=embed_api_endpoint,
    api_key=embed_api_key,
)

In [5]:
print(model)

client=<openai.resources.chat.completions.Completions object at 0x000002EFD7CE4C10> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002EFD8918090> root_client=<openai.lib.azure.AzureOpenAI object at 0x000002EFD89B18D0> root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x000002EFD8951F50> temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********') disabled_params={'parallel_tool_calls': None} azure_endpoint='https://genaiapimna.jnj.com/openai-chat' deployment_name='gpt-35-turbo-16k' openai_api_version='2023-05-15' openai_api_type='azure'


In [6]:
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage(content="Cuentame un chiste para niños")
]

model.invoke(messages)

AIMessage(content='Claro, aquí tienes uno:\n\n¿Qué hace una abeja en el gimnasio?\n\n¡Zum-ba!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 15, 'total_tokens': 41, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-3e8e9893-c57e-4fb1-bc4c-1009545cec42-0', usage_metadata={'input_tokens': 15, 'output_tokens': 26, 'total_tokens': 41, 'input_token_details': {}, 'output_token_details': {}})

## Chains and Output parser

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

messages = [
    SystemMessage(content="Cuentame un chiste para niños")
]

# with chains:
chain = model | parser
print(chain.invoke(messages))


Claro, aquí tienes uno:

¿Qué hace una abeja en el gimnasio?

¡Zum-ba!


## Knowledge cutoffs

In [8]:
messages = [
    SystemMessage(content="Quando foi o Donald trump condenado?")
]
chain = model | parser
print(chain.invoke(messages))

Donald Trump não foi condenado criminalmente durante seu mandato como presidente dos Estados Unidos. No entanto, ele enfrentou dois processos de impeachment na Câmara dos Representantes em 2019 e 2021, mas foi absolvido pelo Senado em ambas as ocasiões.


## Hallucinations

In [9]:
messages = [
    SystemMessage(content="Give me some papers on the relationship between homeschooling and neuroplasticity")
]
chain = model | parser
print(chain.invoke(messages))

1. "Neuroplasticity and Homeschooling: Exploring the Impact of Individualized Education on Brain Development" by Sarah Johnson and Emily Thompson. This paper examines the relationship between homeschooling and neuroplasticity, exploring how individualized education in a homeschooling environment can potentially enhance brain development and promote neuroplasticity.

2. "The Effects of Homeschooling on Neuroplasticity: A Review of Current Research" by Michael Davis and Jennifer Smith. This review paper provides an overview of current research on the effects of homeschooling on neuroplasticity. It discusses the potential benefits of homeschooling in promoting neuroplasticity and highlights the need for further investigation in this area.

3. "Neuroplasticity and Homeschooling: A Case Study Approach" by Laura Miller and Mark Wilson. This paper presents a case study approach to examine the relationship between homeschooling and neuroplasticity. It explores the experiences of homeschooling 

## ChatPromptTemplate 
System + User templates 

In [10]:

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate


system_template = "Translate the following into {language}:"
user_template = "{text}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), 
     ("user", user_template)
     ]
)

chain = prompt_template | model | parser

print(chain.invoke({"language": "italian", "text": "hi!"}))


Ciao!


## Chatbot

In [12]:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


system_template = "You are a helpful assistant. Answer all questions to the best of your ability."
prompt = ChatPromptTemplate.from_messages(
    [ ("system", system_template,),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model | parser


chat_history_chain = [HumanMessage(content="Hey there! I'm Nemo."),
 SystemMessage(content='Hello!'),
 HumanMessage(content='what is my name?')]

In [13]:

print(chain.invoke({"messages": chat_history_chain}))

Your name is Nemo.


## Summarization

In [14]:
review_1 = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""


review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]

prompt = """
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words and in Spanish. 

    Review: ```{review}```
    """

prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt)
    ])


for i in range(len(reviews)):
    inputs={"review":reviews[i]}
    prompt_template.format_messages(**inputs)
    
    chain = prompt_template | model | parser

    response = chain.invoke(inputs)
    print(i, response, "\n")


0 El juguete de panda es suave y lindo, pero un poco pequeño para el precio. Llegó antes de lo esperado. 

1 Lámpara con almacenamiento adicional, buen precio y excelente servicio al cliente. 

2 El cepillo de dientes eléctrico tiene una buena duración de batería, pero la cabeza es demasiado pequeña. Bueno por $50. 

3 El sistema de 17 piezas estaba en oferta en noviembre, pero los precios subieron en diciembre. La calidad ha disminuido y el motor se dañó después de un año. 



In [15]:
prompt = """
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words, also specify if the review is positive, negative or neutral. 
    Give the answers in Spanish. 

    Review: ```{review}```
    """

prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt)
    ])


for i in range(len(reviews)):
    inputs={"review":reviews[i]}
    prompt_template.format_messages(**inputs)
    
    chain = prompt_template | model | parser

    response = chain.invoke(inputs)
    print(i, response, "\n")


0 La reseña es positiva. El juguete de panda es suave y lindo, pero un poco pequeño para el precio. Llegó antes de lo esperado. 

1 Neutral: La lámpara tiene almacenamiento adicional y un precio razonable. El servicio al cliente es excelente. 

2 El revisor está satisfecho con la duración de la batería, pero no le gusta el tamaño de la cabeza del cepillo. En general, considera que es una buena oferta. Positivo. 

3 El revisor menciona que el precio del sistema aumentó después de la venta, pero la calidad ha disminuido. Neutral. 

